# DECISION TREE 

#### Abhinav Bohra 

## 1994 Census database
#### Prediction task is to determine whether a person makes over 50K a year.

# **Decision Tree Class**

In [ ]:
#Data Structure to store tree
class tree_node:
    def __init__(self, attribute, label, subAttributes):
        self.attribute = attribute                                    #Node representing Question
        self.label = label                                            #Label Class (<=50K or >50K)
        self.subAttributes = subAttributes                            #Edges representing possible attribute values of node
        self.child = []                                               #List storing Children nodes of main node
        
# Class containg Decision Tree and all related functions        
class decisionTree:
    
    def __init__(self, max_depth=None):
        self.max_depth = max_depth if max_depth is not None else 10     #max_depth default value =10
        self.max_depth = self.max_depth if max_depth != -1 else 100     #set max_depth value very high to allow full growth
        self.root      = None
        self.attributes = []
        
    #-------------------------------------------------------------------------------------------------       
    #Function to compute the entropy for data-set
    def findEntropy(self,countYes, countNo):
        ratio_yes, ratio_no = countYes/(countYes + countNo), countNo/(countYes + countNo)
        if not (ratio_yes and ratio_no):
            return 0
        else:
            return -ratio_yes * np.log(ratio_yes) - ratio_no * np.log(ratio_no)
    
    #Function to find the Best Classifier Attribute
    def findBestAttribute(self,trainX, trainY, attributes, entropy):
        
        gains = []
        
        for index in range(len(attributes)):
            total_entropy , total_population = 0, 0        
            for indexY in np.unique(trainX[:,index]):            
                temp = np.where(trainX == indexY)[0]            
                countYes = ( trainY[temp]== ">50K"  ).sum()
                countNo  = ( trainY[temp]== "<=50K" ).sum()
                newEntropy = self.findEntropy(countYes, countNo)
                total_entropy += temp.shape[0]*newEntropy
                total_population += temp.shape[0]
 
            total_entropy /= total_population
            gains.append(entropy - total_entropy)
 
        return gains.index(max(gains))
 
    #ID3 Algorithm
    def train(self,trainX, trainY, attributes,depth):
        countYes = ( trainY == ">50K"  ).sum()
        countNo  = ( trainY == "<=50K" ).sum()
        
        if not countYes:
            return tree_node(None, "<=50K", None)
 
        elif not countNo:
            return tree_node(None, ">50K", None)
 
        elif not attributes.shape[0] or depth >= self.max_depth:
            if countYes > countNo:
                return tree_node(None, ">50K", None)
            return tree_node(None, "<=50K", None)
        
        depth = depth + 1        
        entropy = self.findEntropy(countYes, countNo)
        bestIndex = self.findBestAttribute(trainX, trainY, attributes, entropy)
        bestAttribute = attributes[bestIndex]
        subAttributes = np.unique(trainX[:,bestIndex])
        node = tree_node(bestAttribute, None, subAttributes)
        
        for attribute in subAttributes:
 
            temp = np.where( trainX == attribute )[0]
            newTrainX = np.delete(trainX[temp], bestIndex, 1)
            newTrainY = trainY[temp]
            newAttributes = np.delete(attributes, bestIndex)
            node.child.append(self.train(newTrainX, newTrainY, newAttributes,depth))
 
        return node    
    
    #-------------------------------------------------------------------------------------------------------  
    def fitTree(self,X_train,Y_train,attributes):
       
        self.attributes = attributes
        
        self.root = self.train(X_train,Y_train, np.array(self.attributes),0)
    
    #-------------------------------------------------------------------------------------------------------
    #Function to classify datapoint into target class
    def predict(self,x,node):
        if not node.attribute:
            return node.label
        else:
            ind   = np.where(node.attribute == np.array(self.attributes))[0]
            edge  = x[ind]
            index = np.where(edge[0] == np.array(node.subAttributes))[0]

            if(len(index) == 0 ):
                return "<=50K"         #this is the case when an edge was never seen in training data 

            nextNode = node.child[index[0]]
            return self.predict(x,nextNode)

    #Function to fetch predictions
    def getPredictions(self,X_test):
        preds=[]
        for x in X_test:
            preds.append( self.predict(x,self.root) )
            
        return preds
    
    #-------------------------------------------------------------------------------------------------------
    #Function to compute accuracy
    def getAccuracy(self,preds,Y_test):
        numCorrect = 0
        for i in range(0,len(Y_test)):
            if(preds[i]==Y_test[i]):
                numCorrect = numCorrect + 1

        return(numCorrect/len(Y_test))
    
    #-------------------------------------------------------------------------------------------------------

### Helper Functions

###  setData : **Function to Split Data into Train-set, Validation-set, Test-set**

In [ ]:
def setData(data,target_var,splitRatio1=None,splitRatio2=None): 
    '''
    Takes dataframe and target variable name as input and splits into X and Y sets using target_var
    Futher splits X into X_train , X_CV, X_test
    Futher splits Y into Y_train , Y_CV, Y_test
    Splitting done as per input split ratios
    Default Split Ration -> [train : CV : test] = [64 : 16 : 20]
    return attributes (column name) of set X and X_train,X_test,X_CV,Y_train,Y_test,Y_CV as numpy array
    '''
    Y = data[target_var].reset_index(drop=True)
    X =  data.copy(deep=True)
    X.drop([target_var], axis='columns', inplace=True)
    
    if(X is None or Y is None):
        print("Error : X or Y cannot be of NoneType. ")
        return
    
    if(len(X)!= len(Y)):
        print("Error : X and Y should be of same dimension. ")
        return
        
    if(splitRatio1 is not None and (splitRatio1 <= 0 or splitRatio1>= 1) ):
        print("Error : splitRatio1 should be between 0 and 1. ")
        return
                      
    if(splitRatio2 is not None and (splitRatio2 <= 0 or splitRatio2>= 1) ):
        print("Error : splitRatio2 should be between 0 and 1. ")
        return
            
    splitRatio1 = splitRatio1 if splitRatio1 is not None else 0.64     #splitRatio1 default value = 0.64
    splitRatio2 = splitRatio2 if splitRatio2 is not None else 0.16     #splitRatio1 default value = 0.16
    
        
    #Split into trainset, cross validation set and testset        
    
    cutIndex1 = int (splitRatio1*len(X))
    cutIndex2 = cutIndex1 + int (splitRatio2*len(X))

    X_train = X[:cutIndex1]
    X_CV    = X[cutIndex1: cutIndex2]
    X_test  = X[cutIndex2:]

    Y_train = Y[:cutIndex1]
    Y_CV    = Y[cutIndex1: cutIndex2]
    Y_test  = Y[cutIndex2:]

    attributes = X_train.columns
    return attributes,X_train.to_numpy(),  X_test.to_numpy(),  X_CV.to_numpy(),  Y_train.to_numpy(),  Y_test.to_numpy(),  Y_CV.to_numpy()

## showTree:  **Function to print Tree**

In [ ]:
def showTree(root,level):
    if not root.attribute:
        print(root.label)
    
    for ind, child in enumerate(root.child):
        if child.attribute:
            print("\t"*level + root.attribute + " = " + root.subAttributes[ind] + "\n" , end = '')     
        else:
            print("\t"*level + root.attribute + " = " + root.subAttributes[ind] + ":- " , end = ' ')
        
        showTree(child,level+1) 

## pruneTree: **Function to Prune Tree**

In [ ]:
'''
        Statsitical Test Used : Reduced Error Pruning

'''
def pruneTree(node,validationSet_X, validationSet_Y):  
    #print(node.attribute)
    if not node.attribute:    #if node is a label, pruning cannot be done, return
        return
    
    if len(validationSet_X) == 0:
        return
    
    #Check accuracy before pruning
    preds_before_pruning    = myDT.getPredictions(validationSet_X)
    accuracy_before_pruning = myDT.getAccuracy(preds_before_pruning,validationSet_Y)
    
    #Store node data temporarily
    temp_node       = tree_node("none","none","none")
    temp_node.attribute=node.attribute
    temp_node.label=node.label
    temp_node.child = node.child
    temp_node.subAttributes=node.subAttributes
    
    #Prune node
    node.attribute=None
    node.subAttributes=None
    node.child=[]
    
    countYes = ( validationSet_X == ">50K"  ).sum()
    countNo  = ( validationSet_Y == "<=50K" ).sum()

    if countYes > countNo:
        node.label=">50K"
    else:
        node.label="<=50K"
       
    #Check accuracy after pruning
    
    preds_after_pruning    = myDT.getPredictions(validationSet_X)
    accuracy_after_pruning = myDT.getAccuracy(preds_after_pruning,validationSet_Y)
    
    if accuracy_after_pruning >= accuracy_before_pruning:
        return
    
    else:
      #Copy initial node
        node.label         = temp_node.label
        node.attribute      = temp_node.attribute
        node.subAttributes = temp_node.subAttributes
        node.child         = temp_node.child

        for child in node.child:
            pruneTree(child,validationSet_X, validationSet_Y) 

## fetchData:  **Function to fetch data from directory and perform data pre-processing**

In [ ]:
def fetchData():
    #Setting up Training Dataset
    data = pd.read_csv('dataset.data', sep=", ", header=None,engine='python')
    data.columns =['age','workclass','fnlwgt','education','education-num','marital-status','occupation',
    'relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country','target']

    #Data Preprocessing
    def fillMissingVal(x,mode):
        if x == '?':
            return mode
        else:
            return x

    def splitVar(x,threshold):
        if x <= threshold:
            return "<="+ str(threshold)
        else:
            return ">" + str(threshold)

    #Fill Missing Values    
    for col in data.keys()[:-1]:
        mode = (data[col].mode()[0]) 
        data[col]=data.apply(lambda x: fillMissingVal(x[col],mode),axis=1)
            
    #Convert Continous Varaibles into Discrete Variables using split threshold    
    age_threshold=37
    data['age']=data.apply(lambda x: splitVar(x['age'],age_threshold),axis=1)
    
    fnlwgt_threshold=178300
    data['fnlwgt']=data.apply(lambda x: splitVar(x['fnlwgt'],fnlwgt_threshold),axis=1)

    # Dropping few variables after data analysis
    data.drop(['capital-gain', 'capital-loss','hours-per-week','education-num'], axis='columns', inplace=True)

    return data

In [ ]:
'''
PART 1

Build a decision tree by taking as input a maximum depth and by randomly splitting the dataset as 80/20 split
and provide the accuracy by averaging over 10 random 80/20 splits. Consider that particular tree which provides 
the best test accuracy as the desired one. 

'''

def findBestTree(maxDepth):
    accSum=0
    max=0
    best_Dataset=data
    for i in range (0,10):
        temp_data=data.sample(frac=1)
        #Instantiate Decision Tree with input depth
        myDT  = decisionTree(maxDepth)
        #Set Training, crossvalidation and  Testing Data
        attributes,X_train,X_test,X_CV,Y_train,Y_test,Y_CV = setData(temp_data,'target')
        #Fit Tree to training data
        myDT.fitTree(X_train,Y_train,attributes)
        #Fetch predictions
        preds = myDT.getPredictions(X_test)
        #Calculate Accuracy
        acc   = myDT.getAccuracy(preds,Y_test)   
        #Destroy Instance
        del myDT     
        #Store Best Dataset split
        accSum = accSum + acc
        if(acc > max):
            max = acc
            best_Dataset=temp_data

    #Calculate Average Accuracy     
    accAvg =accSum/10
    return accAvg, max, best_Dataset

In [ ]:
'''
PART 2 : Best possible depth limit to be used for your dataset 
'''
import matplotlib.pyplot as plt 

def findBestDepth(bestDataset):
    AccuracyTrain =[]
    AccuracyTest  =[]
    x=[]
    maxD=0
    maxAcc =0
    for depth in range (1,11):
        #Fetch Dataset
        fetchData()
        #Instantiate Decision Tree with input depth
        myDT1  = decisionTree(depth)
        #Set Training, crossvalidation and  Testing Data
        attributes,X_train,X_test,X_CV,Y_train,Y_test,Y_CV = setData(best_Dataset,'target')
        #Fit Tree to training data
        myDT1.fitTree(X_train,Y_train,attributes)
        
        preds = myDT1.getPredictions(X_train)
        acctrain   = myDT1.getAccuracy(preds,Y_train)
        AccuracyTrain.append(acctrain)
        
        preds1    = myDT1.getPredictions(X_test)
        acctest  = myDT1.getAccuracy(preds1,Y_test)
        AccuracyTest.append(acctest)
        
        if(maxAcc < acctest):
            maxAcc = acctest
            maxD   = depth

        x.append(depth)

    plt.plot(x, AccuracyTrain,label="Train")
    plt.plot(x, AccuracyTest ,label="Test" ) 
    plt.legend()
    plt.xlabel('Depth') 
    plt.ylabel('Accuracy') 
    plt.title('Decision Tree Depth Analysis') 
    plt.show()  
    return maxD

# Main Driver Function

In [ ]:
import pandas as pd
import numpy as np
#Fetch Dataset
data = fetchData()

### Part 1 of assignment

In [ ]:
'''
      PART 1 : Provide the accuracy by averaging over 10 random splits. Consider that particular tree 
      which provides the best test accuracy as the desired one. 

      avgAcc stores the accuracy by averaging over 10 random dataset splits
      maxAcc stores the max accuracy among all the 10 splits
      best_Dataset stores the dataset which gave maximum accuracy (Tree for this dataset is built again in later steps)
'''

max_Depth = 3 
# Change this variable to change input maxnimum depth
# Enter -1 to allow full growth of tree

avgAcc, maxAcc, best_Dataset = findBestTree(max_Depth)
print("Maximum Accuracy is " + str(maxAcc))
print("Average Accuracy is " + str(avgAcc))

### Part 2 of assignment

In [ ]:
'''
      PART 2 :  Find Best possible depth limit to be used for your dataset. 
      Provide a plot explaining the same.

      Function findBestDepth makes tree with maxDepth varying from 1 to 10 using the best_Dataset 
      obtained from previous steps
'''

maxD = findBestDepth(best_Dataset)   
print("Maximium Depth is : " +str(maxD))

### Part 3 of assignment

In [ ]:
'''
  PART 3: Perform the pruning operation over the tree obtained in question 2 using a valid statistical test for comparison. 
  
  Function Prune Tree perfoms reduced error post pruning on tree myDT obtained from PART 2
  myDT gets pruned and updated accordingly
'''
#Build tree with best max depth and best Dataset split
myDT  = decisionTree(maxD)
#Set Training, crossvalidation and  Testing Data
attributes,X_train,X_test,X_CV,Y_train,Y_test,Y_CV = setData(best_Dataset,'target')
#Fit Tree to training data
myDT.fitTree(X_train,Y_train,attributes)

preds = myDT.getPredictions(X_test)
acc_before_pruning   = myDT.getAccuracy(preds,Y_test)
        
#Prune Tree
pruneTree(myDT.root,X_CV,Y_CV)

preds_new = myDT.getPredictions(X_test)
acc_after_pruning  = myDT.getAccuracy(preds_new,Y_test)

print("Accuracy before pruning : " + str(acc_before_pruning))
print("Accuracy after pruning : "  + str(acc_after_pruning ))


### Part 4 of assignment

In [ ]:
'''
  PART 4 : Print the final decision tree obtained from question 3 following the hierarchical levels of data attributes as nodes of the tree

  Function showTree prints the pruned tree myDT obtained from PART 3 in the desired format
'''

#Print Pruned Decision Tree
print("Final Tree after Post Pruning")
showTree(myDT.root,0)